Sentiment analysis - Colab in 수현's

In [ ]:
'''
sentiment anaylsis

limit: over 512 token truncate --> long bert sentiment anaylsis 찾고 해야함

input: helpful_vote 포함한 원본데이터
output: sentiment analysis 열이 추가된 데이터

'''


import pandas as pd
from transformers import pipeline, BertTokenizer, BertForSequenceClassification

# Load the review data
file_path = "/content/resnet_helpful_processed.csv"
df = pd.read_csv(file_path)

# Initialize the sentiment analysis pipeline with a more popular model
model_name = 'nlptown/bert-base-multilingual-uncased-sentiment'
tokenizer = BertTokenizer.from_pretrained(model_name)
model = BertForSequenceClassification.from_pretrained(model_name)
sentiment_analysis = pipeline("sentiment-analysis", model=model, tokenizer=tokenizer)

# Function to truncate text using the tokenizer
def truncate_text(text, max_length=512):
    if isinstance(text, float):
        text = ""
    encoded_input = tokenizer(text, truncation=True, max_length=max_length, return_tensors='pt')
    truncated_text = tokenizer.decode(encoded_input['input_ids'][0], skip_special_tokens=True)
    return truncated_text

# Convert all entries to string and truncate texts over 512 tokens
df['review_text'] = df['review_text'].astype(str)
df['truncated_review_text'] = df['review_text'].apply(lambda x: truncate_text(x, max_length=512))

# Apply sentiment analysis to the truncated review text
df['sentiment'] = df['truncated_review_text'].apply(lambda x: sentiment_analysis(x)[0]['label'])
df['sentiment_score'] = df['truncated_review_text'].apply(lambda x: sentiment_analysis(x)[0]['score'])

# Save the result to a new CSV file
output_file_path = "/content/resnet_helpful_sentiment_processed2.csv"
df.to_csv(output_file_path, index=False)

print(f"Processed data saved to {output_file_path}")